In [1]:
import pandas as pd
import numpy as np
import models
import itertools
import cPickle
import gensim
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
with open("./ckd/results.dat", 'rb') as infile:
    results = cPickle.load(infile)

In [7]:
results.keys()

['rfc_ccs',
 'lr_ccs',
 'rnn_raw',
 'rfc_raw',
 'crf',
 'lr_lda',
 'rnn_ccs',
 'lr_raw',
 'rfc_lda']

In [18]:
disease_name = "ckd"
path_to_data = "./diabetes_data.csv"
# load the data from csv
data = pd.read_csv(path_to_data, index_col=0).dropna()
col = {"Diseased":"Diseased",
        "FollowUp":"FollowUp",
        "Codes":"ObsCodes"}
# create train, validation, and test sets -- change this section for different dataset sizes
dis_ids = np.random.permutation(data[(data[col["Diseased"]]==1)&(data[col["FollowUp"]]>15)].index)
ctl_ids = np.random.permutation(data[(data[col["Diseased"]]==0)&(data[col["FollowUp"]]>15)].index)
N_dis = len(dis_ids)
N_dis_train = int(.7*N_dis)
N_dis_valid = int(.15*N_dis)
# next three lines are for if you want number of control cases to be a fixed factor greater than diseased
#factor = 10
#N_ctl_train = N_dis_train*factor
#N_ctl_valid = N_dis_valid*factor
#N_ctl_test = N_dis_valid*factor
# next three lines are for if you want to use all available control cases
#N_ctl = len(ctl_ids)
#N_ctl_train = int(.5*N_ctl)
#N_ctl_valid = int(.25*N_ctl)
N_ctl = 100000
N_ctl_train = int(.7*N_ctl)
N_ctl_valid = int(.15*N_ctl)
N_ctl_test = int(.15*N_ctl)
data["Train"] = -1
data.loc[dis_ids[:N_dis_train],"Train"] = 1 
data.loc[dis_ids[N_dis_train:N_dis_train+N_dis_valid],"Train"] = 2 
data.loc[dis_ids[N_dis_train+N_dis_valid:],"Train"] = 0
data.loc[ctl_ids[:N_ctl_train],"Train"] = 1
data.loc[ctl_ids[N_ctl_train:N_ctl_train+N_ctl_valid],"Train"] = 2
data.loc[ctl_ids[N_ctl_train+N_ctl_valid:N_ctl_train+N_ctl_valid+N_ctl_test],"Train"] = 0

In [23]:
data[["ObsCodes","Span","FollowUp"]]

,ObsCodes,Span,FollowUp
0,"V0381,V0489,7746,6910,7742,7746,V0382,V202,V05...",-421.0,-421
1,V202,0.0,0
2,"38200,V202,V202",52.0,52
3,"V053,V068,V202,38200,V0389,V202,V0389,V054",-189.0,-189
4,"V202,7746,46619,7746,7862,V202,76520,V773,7652...",-551.0,-551
6,"V202,78609,V0382,V0489,V0382,V0381,74363",-84.0,-84
8,"V0481,V3000,7824",-259.0,-259
10,"V0381,7746,V3000,V202",-98.0,-98
12,"V2031,V3000,53081,3829,V3000,7746,V502",443.0,52
13,"5991,76527,76514,7746,28730,76514,28730,76527,...",-424.0,-424


In [32]:
# initialize global params
disease_name = "ckd"
path_to_data = "./diabetes_data.csv"
path_to_ccs = "./ccs_map.txt"
path_to_save_folder = "./{}/".format(disease_name)
col = {"Diseased":"Diseased",
        "FollowUp":"FollowUp",
        "Codes":"ObsCodes"}

##################################################
################ DATA PREPARATION ################
##################################################

# load the data from csv
data = pd.read_csv(path_to_data, index_col=0).dropna()

# create train, validation, and test sets -- change this section for different dataset sizes
dis_ids = np.random.permutation(data[(data[col["Diseased"]]==1)&(data[col["FollowUp"]]>30)].index)
ctl_ids = np.random.permutation(data[(data[col["Diseased"]]==0)&(data[col["FollowUp"]]>30)].index)
N_dis = len(dis_ids)
N_dis_train = int(.5*N_dis)
N_dis_valid = int(.25*N_dis)
# next three lines are for if you want number of control cases to be a fixed factor greater than diseased
factor = 10
N_ctl_train = N_dis_train*factor
N_ctl_valid = N_dis_valid*factor
N_ctl_test = N_dis_valid*factor
# next three lines are for if you want to use all available control cases
#N_ctl = len(ctl_ids)
#N_ctl_train = int(.5*N_ctl)
#N_ctl_valid = int(.25*N_ctl)
data["Train"] = -1
data.loc[dis_ids[:N_dis_train],"Train"] = 1 
data.loc[dis_ids[N_dis_train:N_dis_train+N_dis_valid],"Train"] = 2 
data.loc[dis_ids[N_dis_train+N_dis_valid:],"Train"] = 0
data.loc[ctl_ids[:N_ctl_train],"Train"] = 1
data.loc[ctl_ids[N_ctl_train:N_ctl_train+N_ctl_valid],"Train"] = 2
data.loc[ctl_ids[N_ctl_train+N_ctl_valid:N_ctl_train+N_ctl_valid+N_ctl_test],"Train"] = 0

# create raw tokenized data
dictionary   = models.create_dictionary(data.loc[data.Train==1,col["Codes"]].values)
train_tokens = models.create_tokens(data.loc[data.Train==1,col["Codes"]].values, dictionary)
valid_tokens = models.create_tokens(data.loc[data.Train==2,col["Codes"]].values, dictionary)
test_tokens  = models.create_tokens(data.loc[data.Train==0,col["Codes"]].values, dictionary)

# create ccs tokenized data
ccs_map = models.create_ccs_map(path_to_ccs)
unknown_codes = set(dictionary.keys()).difference(set(ccs_map.keys()))
p = len(ccs_map.values())
for c in unknown_codes:
    ccs_map[c] = p
train_ccs_tokens = models.create_tokens(data.loc[data.Train==1,col["Codes"]].values, ccs_map)
valid_ccs_tokens = models.create_tokens(data.loc[data.Train==2,col["Codes"]].values, ccs_map)
test_ccs_tokens  = models.create_tokens(data.loc[data.Train==0,col["Codes"]].values, ccs_map)

# create sparse BOW from raw tokens
sparse_train_data = models.create_sparse_array_from_tokens(train_tokens,len(dictionary)+1)
sparse_valid_data = models.create_sparse_array_from_tokens(valid_tokens,len(dictionary)+1)
sparse_test_data  = models.create_sparse_array_from_tokens(test_tokens,len(dictionary)+1)

# create sparse BOW from ccs tokens  
sparse_ccs_train_data = models.create_sparse_array_from_tokens(train_ccs_tokens,len(ccs_map)+1)
sparse_ccs_valid_data = models.create_sparse_array_from_tokens(valid_ccs_tokens,len(ccs_map)+1)
sparse_ccs_test_data  = models.create_sparse_array_from_tokens(test_ccs_tokens,len(ccs_map)+1)


#################################################
################## EXPERIMENTS ##################
#################################################

results = {}

In [21]:
# RF EXPERIMENT ON CCS TOKENIZED DATA
print "RF CCS experiments"
best_rfc_score = 0
best_rfc_model = None
best_rfc_params = None
for n in [50, 100, 250, 500]:
    params={"n_trees":n}
    score, mod = models.random_forest_model(params, 
                                            sparse_ccs_train_data,
                                            list(data.loc[data.Train==1,col["Diseased"]].values),
                                            sparse_ccs_valid_data, 
                                            list(data.loc[data.Train==2,col["Diseased"]].values))
    if score > best_rfc_score:
        best_rfc_score=score
        best_rfc_model=mod
        best_rfc_params=params
pred_rfc = best_rfc_model.predict_proba(sparse_ccs_test_data)[:,1]
results["rfc_ccs"] = [pred_rfc, best_rfc_params]

RF CCS experiments


KeyboardInterrupt: 

In [22]:
# LR EXPERIMENT ON RAW TOKENIZED DATA
print "LR raw experiments"
best_lr_score = 0
best_lr_model = None
best_lr_params = None
for c in [.1, .5, 1.0, 5.0]:
    params={"C":c}
    score, mod = models.logistic_regression_model(params, 
                                                  sparse_train_data,
                                                  list(data.loc[data.Train==1,col["Diseased"]].values),
                                                  sparse_valid_data, 
                                                  list(data.loc[data.Train==2,col["Diseased"]].values))
    if score > best_lr_score:
        best_lr_score=score
        best_lr_model=mod
        best_lr_params=params
pred_lr = best_lr_model.predict_proba(sparse_test_data)[:,1]
results["lr_raw"] = [pred_lr, best_lr_params]

LR raw experiments


In [23]:
# CRF EXPERIMENTS
print "CRF experiments"
best_crf_score = 0
best_crf_model = None
best_crf_params = None
y_valid_crf = data.loc[data.Train==2,col["Diseased"]].values
for w in [0,1,3]:
    params = {'w':w}
    x_train = models.create_crf_tokens(data.loc[data.Train==1,col["Codes"]].values,
                                       ccs_map,
                                       w)
    x_valid = models.create_crf_tokens(data.loc[data.Train==2,col["Codes"]].values,
                                       ccs_map,
                                       w)
    y_train_crf = [[str(lab)]*len(seq) for seq,lab 
                   in itertools.izip(x_train,data.loc[data["Train"]==1,col["Diseased"]].values)]
    print "training"
    score, mod = models.conditional_random_field_model(params,
                                                       x_train,
                                                       y_train_crf,
                                                       x_valid,
                                                       y_valid_crf)
    if score > best_crf_score:
        best_crf_score = score
        best_crf_model = mod
        best_crf_params = params  
x_test = models.create_crf_tokens(data.loc[data.Train==0,col["Codes"]].values,
                                  ccs_map,
                                  best_crf_params['w'])
marg = best_crf_model.predict_marginals(x_test)
pred_crf = [r[-1]['1'] for r in marg]
results["crf"] = [pred_crf, best_crf_params]

CRF experiments


In [29]:
truth = data.loc[data.Train==0,col["Diseased"]]
for mod in results.keys():
    fpr, tpr, thresh = roc_curve(truth, results[mod][0], pos_label=1)
    score = roc_auc_score(truth, results[mod][0])
    plt.plot(fpr, tpr, 'b', label="{}({:.3f})".format(mod,score), color=np.random.rand(3,1))
plt.plot([0,1], '--', color='.45')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('ROC Curves')
plt.show()


In [28]:
print "LDA experiments"
best_lr_lda_model = None
best_lr_lda_score = 0
best_lr_gensim_model = None
best_lr_lda_params = None

best_rfc_lda_model = None
best_rfc_lda_score = 0
best_rfc_gensim_model = None
best_rfc_lda_params = None

gensim_d = gensim.corpora.dictionary.Dictionary([c.split(",") for c in data.loc[data.Train==1,col["Codes"]]])
gensim_train_corp = [gensim_d.doc2bow(c.split(",")) for c in data.loc[data.Train==1, col["Codes"]]]
gensim_valid_corp = [gensim_d.doc2bow(c.split(",")) for c in data.loc[data.Train==2, col["Codes"]]]
gensim_test_corp = [gensim_d.doc2bow(c.split(",")) for c in data.loc[data.Train==0, col["Codes"]]]
for n_tops in [10,30,50,100]:
    gensim_mod = gensim.models.ldamodel.LdaModel(corpus=gensim_train_corp,
                                          id2word=gensim_d,
                                          num_topics=n_tops, 
                                          #workers=1,
                                          chunksize=10,
                                          eval_every=0)

    # create low-dimensional vectors
    lda_train_vectors = [[t[1] for t in gensim_mod.__getitem__(c, eps=0)] 
                                   for c in gensim_train_corp]
    lda_valid_vectors = [[t[1] for t in gensim_mod.__getitem__(c, eps=0)] 
                                   for c in gensim_valid_corp]

    for n in [50, 100, 250, 500]:
        params={"n_trees":n, "n_topics":n_tops}
        score, mod = models.random_forest_model(params, 
                                                lda_train_vectors,
                                                list(data.loc[data.Train==1,"Diseased"].values),
                                                lda_valid_vectors, 
                                                list(data.loc[data.Train==2,"Diseased"].values))
        if score > best_rfc_lda_score:
            best_rfc_lda_score=score
            best_rfc_lda_model=mod
            best_rfc_lda_params=params
            best_rfc_gensim_model = gensim_mod
    
    for c in [.1, .5, 1.0, 5.0]:
        params={"C":c, "n_topics":n_tops}
        score, mod = models.logistic_regression_model(params, 
                                                      lda_train_vectors,
                                                      list(data.loc[data.Train==1,"Diseased"].values),
                                                      lda_valid_vectors, 
                                                      list(data.loc[data.Train==2,"Diseased"].values))
        if score > best_lr_lda_score:
            best_lr_lda_score=score
            best_lr_lda_model=mod
            best_lr_lda_params=params
            best_lr_gensim_model = gensim_mod
            
# predict using the best models
lda_test_vectors = [[t[1] for t in best_rfc_gensim_model.__getitem__(c, eps=0)] 
                    for c in gensim_test_corp]
pred_rfc_lda = best_rfc_lda_model.predict_proba(lda_test_vectors)[:,1]
results["rfc_lda"] = [pred_rfc_lda, best_rfc_lda_params]

lda_test_vectors = [[t[1] for t in best_lr_gensim_model.__getitem__(c, eps=0)] 
                    for c in gensim_test_corp]
pred_lr_lda = best_lr_lda_model.predict_proba(lda_test_vectors)[:,1]
results["lr_lda"] = [pred_lr_lda, best_lr_lda_params]

LDA experiments


In [33]:
data.loc[data.Diseased==1].shape

(7094, 14)